# <span style="color: green;">Cross-correlating GW and Galaxy catalogs</span>

# <span style="color: red;">Questions:</span>

<ol>
  <li>Cell 11 - Shouldn't put the ra function instead of rr to actually calculate the spherical harmonics in the desired scale?</li>
  <li>Markdown spectra cell - How does all this calculation remains meaningful when we're marginalizing over all of the possible values of bias for BHs and GW? It looks like a cycle! the only difference between the cross spectras and the auto spectras arises from their difference in the bias value at the different r!!!</li>
</ol>

In [1]:
using TwoFAST
using Dierckx
using DelimitedFiles
using QuadGK
using LinearAlgebra
using SpecialFunctions
using SparseArrays

# Loading the power spectrum to be projected onto the spherical harmonics:

In [2]:
path = homedir() * "/.julia/packages/TwoFAST/"
path *= readdir(path)[1]
path *= "/test/data/planck_base_plikHM_TTTEEE_lowTEB_lensing_post_BAO_H070p6_JLA_matterpower.dat"
d = readdlm(path, comments=true)
pk = Spline1D(d[:,1], d[:,2])


N = 4096    # number of points to use in the Fourier transform
kmax = 1e3  # maximum k-value
kmin = 1e-5 # minimum k-value
chi0 = 1e-3   # minimum r-value (should be ~1/kmax)
q = 1.1; #What is it exactly?!

#Neal's initial condition:
#N = 4096;
#kmin = 1e-5;
#kmax = 1e3;


# First one may want to calculate the two point correlation function as described below:
## $ \xi_l^v(r) = \int_0^{\infty} \frac{k^2 dk}{2\pi^2} P_k \frac{j_l(kr)}{(kr)^v} $


In [3]:

print("ξ(r), ℓ=0, ν=0: ")
r00, xi00 = xicalc(pk, 0, 0; N=N, kmin=kmin, kmax=kmax, r0=chi0)

print("ξ(r), ℓ=0, ν=-2:")
r, xi0m2 = xicalc(pk, 0, -2; N=N, kmin=kmin, kmax=kmax, r0=chi0)

ξ(r), ℓ=0, ν=0: ξ(r), ℓ=0, ν=-2:

([0.001, 0.0010045073642544625, 0.0010090350448414473, 0.0010135831333340658, 0.0010181517217181819, 0.0010227409023942737, 0.0010273507681793025, 0.0010319814123085885, 0.001036632928437698, 0.0010413054106443369  …  95602.39010953065, 96033.30490535141, 96466.16199111959, 96900.9701214438, 97337.73809039182, 97776.47473167058, 98217.18891880369, 98659.88956530999, 99104.58562488579, 99551.28609158496], [6.231495803216023, 6.284801775671147, 6.338536236235863, 6.392702125504542, 6.447302394647362, 6.502340005240893, 6.557817929092735, 6.613739148057433, 6.670106653850142, 6.726923447851813  …  5.724877996684647, 5.7740885898887, 5.823699126134334, 5.8737124337912725, 5.9241313532169135, 5.974958736657006, 6.026197448103745, 6.077850363183417, 6.129920369008597, 6.182410364052219])

# Now one may want to calculate the $M_{ll}$ which is:

## The final step of projecting onto the spherical harmonics is to actually find the $C_l$ coefficients as follows:


## <span style="color: green;">$C_l(r, r') = \frac{2}{\pi} \int k^2 dk \ P(k) \ j_l(kr) \ j_l(kr')$</span>

## To calculate this one may want to first calculate the fourier transform of the multiplication of two spherical bessel functions:

## $ e^{q\sigma} j_l(\alpha e^{\sigma}) \ j_l'(\beta e^{\sigma}) = \int \frac{dt}{2 \pi} e^{it \sigma} M_{ll'}^{q}(t,R)$

In [4]:
nl=300; ell=[i for i in 1:nl ]; #500 
nR=184; RR = [10^((i-nR)/512) for i in 1:nR ]; #384

#TwoFAST example
#ell = [42]  # only ell=42 for this run
#RR = [0.6, 0.7, 0.8, 0.9, 1.0]

## Next step can be skipped if you already ran it once and the cache file exists

In [5]:
# calculate M_ll at high ell, result gets saved to a file:
#This calculates the multiplication of two besse functions
f21cache = F21EllCache(maximum(ell), RR, N; q=q, kmin=kmin, kmax=kmax, χ0=chi0)
write("out/F21EllCache", f21cache)

# calculate all M_ll, result gets saved to a file:
mlcache = MlCache(ell, "out/F21EllCache", "out/MlCache")
write("out/MlCache", mlcache)

  q=1.1, R=0.43911485066998085:	  0.162066 seconds (229.07 k allocations: 35.594 MiB, 34.71% compilation time)
  q=1.1, R=0.44109410125149434:	  0.137535 seconds (224.06 k allocations: 35.176 MiB, 20.75% gc time)
  q=1.1, R=0.44308227303632963:	  0.125343 seconds (223.84 k allocations: 35.134 MiB, 11.30% gc time)
  q=1.1, R=0.44507940623559955:	  0.123091 seconds (224.91 k allocations: 35.323 MiB, 6.41% gc time)
  q=1.1, R=0.44708554124166333:	  0.118189 seconds (224.20 k allocations: 35.206 MiB, 5.58% gc time)
  q=1.1, R=0.449100718628943:	  0.115656 seconds (224.49 k allocations: 35.246 MiB)
  q=1.1, R=0.45112497915474453:	  0.117691 seconds (224.34 k allocations: 35.223 MiB, 6.72% gc time)
  q=1.1, R=0.4531583637600818:	  0.121376 seconds (224.10 k allocations: 35.181 MiB, 6.00% gc time)
  q=1.1, R=0.4552009135705047:	  0.124167 seconds (224.59 k allocations: 35.268 MiB, 6.18% gc time)
  q=1.1, R=0.4572526698969311:	  0.121768 seconds (224.22 k allocations: 35.209 MiB, 6.50% gc time

  q=1.1, R=0.6406792005704995:	  0.128665 seconds (219.79 k allocations: 34.429 MiB, 6.08% gc time)
  q=1.1, R=0.6435669750977286:	  0.131208 seconds (219.91 k allocations: 34.450 MiB, 6.93% gc time)
  q=1.1, R=0.6464677658766367:	  0.131157 seconds (219.50 k allocations: 34.374 MiB, 5.71% gc time)
  q=1.1, R=0.6493816315762113:	  0.130292 seconds (219.44 k allocations: 34.360 MiB, 5.85% gc time)
  q=1.1, R=0.6523086311298825:	  0.117492 seconds (219.30 k allocations: 34.337 MiB)
  q=1.1, R=0.6552488237367147:	  0.128688 seconds (219.52 k allocations: 34.379 MiB, 7.55% gc time)
  q=1.1, R=0.6582022688626041:	  0.130757 seconds (219.29 k allocations: 34.335 MiB, 5.29% gc time)
  q=1.1, R=0.6611690262414815:	  0.130858 seconds (219.61 k allocations: 34.393 MiB, 5.72% gc time)
  q=1.1, R=0.6641491558765202:	  0.126063 seconds (219.22 k allocations: 34.325 MiB, 5.62% gc time)
  q=1.1, R=0.6671427180413495:	  0.120316 seconds (219.28 k allocations: 34.336 MiB)
  q=1.1, R=0.670149773281274:	

  q=1.1, R=0.9389798010476961:	  0.275390 seconds (225.34 k allocations: 34.090 MiB, 2.89% gc time)
  q=1.1, R=0.9432121250386007:	  0.292848 seconds (225.32 k allocations: 34.085 MiB)
  q=1.1, R=0.9474635256553754:	  0.275674 seconds (225.34 k allocations: 34.092 MiB, 2.43% gc time)
  q=1.1, R=0.9517340888833214:	  0.322815 seconds (225.34 k allocations: 34.092 MiB, 2.23% gc time)
  q=1.1, R=0.9560239010953075:	  0.348934 seconds (225.35 k allocations: 34.097 MiB, 2.33% gc time)
  q=1.1, R=0.9603330490535164:	  0.423555 seconds (225.38 k allocations: 34.095 MiB, 2.15% gc time)
  q=1.1, R=0.9646616199111993:	  0.407230 seconds (225.41 k allocations: 34.103 MiB)
  q=1.1, R=0.9690097012144389:	  0.407680 seconds (225.34 k allocations: 34.090 MiB, 1.88% gc time)
  q=1.1, R=0.9733773809039202:	  0.467855 seconds (225.46 k allocations: 34.112 MiB, 1.46% gc time)
  q=1.1, R=0.9777647473167089:	  0.500301 seconds (225.40 k allocations: 34.104 MiB, 1.34% gc time)
  q=1.1, R=0.9821718891880378:

ell 135, elapsed: 0.4570000171661377
ell 134, elapsed: 0.5390000343322754
ell 133, elapsed: 1.5149998664855957
ell 132, elapsed: 0.20600008964538574
ell 131, elapsed: 0.19000005722045898
ell 130, elapsed: 0.2070000171661377
ell 129, elapsed: 0.1809999942779541
ell 128, elapsed: 0.10900020599365234
ell 127, elapsed: 0.20099997520446777
ell 126, elapsed: 0.18999981880187988
ell 125, elapsed: 0.18900012969970703
ell 124, elapsed: 0.20199990272521973
ell 123, elapsed: 2.005000114440918
ell 122, elapsed: 0.30100011825561523
ell 121, elapsed: 0.1979999542236328
ell 120, elapsed: 0.13199996948242188
ell 119, elapsed: 0.1529998779296875
ell 118, elapsed: 0.1100001335144043
ell 117, elapsed: 0.1099998950958252
ell 116, elapsed: 0.12400007247924805
ell 115, elapsed: 0.24099993705749512
ell 114, elapsed: 0.11400008201599121
ell 113, elapsed: 0.12100005149841309
ell 112, elapsed: 0.2109999656677246
ell 111, elapsed: 0.437999963760376
ell 110, elapsed: 0.5320000648498535
ell 109, elapsed: 0.4589998

200

## Now here we calculated the multiplication of two bessel functions. This can be used to read directly now and calculate the C_l's the coefficients of the spherical harmonics that we were looking for. 

### The net step is just here to help us smallen our arrays a little bit more by taking a look at the values of r!
## We just want the power spectrum on the scale $\ 1<r<10K\  MPc/h$ 

In [6]:
rr = calcwljj(pk, RR; ell=[-1], kmin=kmin, kmax=kmax, N=N, r0=chi0, q=q);

In [7]:
rr[1537]

0.9999999999999998

In [8]:
rr[3585]

9999.99999999997

In [9]:
length(rr)

4096

In [10]:
#What does i exactly corresponds to here?
#we just want the rs between 1 to 10000Mpc/h
imin=1537;
imax=3585;
#Now we just want to make an array for the C_ls which its size id eqaul to imin to imax 
ra = rr[imin:imax]
nr=length(ra);
wl=zeros(nr,nr,nl);

#As a reminder nl was jsut the size of the ell array

# Actually calculating the $C_l$s

In [11]:
#finding wij's
function outfunc(wjj, ell, rr, RR)
    nR=length(RR);
    l=Int(ell);
    
    #nr is here because we're only interested in the matrix elements between two specific scale and not everything
    
    for i in 1:nr
        jmin=max(1,i+1-nR);
        wl[i,jmin:i,l] = (wjj[1])[i+imin-1,(nR-(i-jmin)):nR];
        wl[jmin:i,i,l] = wl[i,jmin:i,l]';
        #' means adjont
    end
    #end for each for loop or any conditional thing
end

#Now the next will calculate the spherical harmonics with the cache it had made from the multiplication of two bessel 
#functions and then here the outfunc will help us in the term that RR would be the rr array but only from the kmin to kmax
#then we give the power spectrum itself Pk to actually calculate the spherical harmonics for it
#outfunct then will put the input in the array wij as defined

rr = calcwljj(pk, RR; ell=ell, kmin=kmin, kmax=kmax, N=N, r0=chi0, q=q, outfunc=outfunc, cachefile="out/MlCache/MlCache.bin");



# w00 = Array{Float64}(undef, N, length(RR))
# w02 = Array{Float64}(undef, N, length(RR))
# function outfunc(wjj, ell, rr, RR)
#     if ell == 42
#         w00[:,:] = wjj[1]
#         w02[:,:] = wjj[2]
#     end
# end
# rr = calcwljj(pk, RR; ell=ell, kmin=kmin, kmax=kmax, N=N, r0=chi0, q=q, outfunc=outfunc, cachefile="out/MlCache/MlCache.bin")

  0.342538 seconds (31 allocations: 66.000 KiB)
make_phi():   0.001789 seconds (50 allocations: 274.609 KiB)
 19.631860 seconds (8.69 M allocations: 1.955 GiB, 0.99% compilation time)
tskip:       0.000169 sec (0 allocations: 0 byte, 0.0% gc time)
tread:       10.035765 sec (15 allocations: 4.961 KB, 0.0% gc time)
tmultphi:    1.216028 sec (342 allocations: 16.844 KB, 0.0% gc time)
tbrfft:      3.751187 sec (0 allocations: 0 byte, 0.0% gc time)
tmultprefac: 1.122042 sec (0 allocations: 0 byte, 0.0% gc time)
toutfunc:    3.504779 sec (8.682 M allocations: 1.954 GB, 0.0% gc time)


## Now everything above was actually another step to save the result of wl which is our $C_l$ objects and also calculte the scale array. To this point now we have our spherical harmonics and $C_l$ coefficients we just need to apply a binning and calculate the covarinace matrix and the Fisher matrix from it


In [16]:
wl

2049×2049×300 Array{Float64, 3}:
[:, :, 1] =
 15.4627  15.437   15.4034  15.3646  15.3219  …   0.0          0.0
 15.437   15.4226  15.3969  15.3634  15.3247      0.0          0.0
 15.4034  15.3969  15.3826  15.3569  15.3234      0.0          0.0
 15.3646  15.3634  15.3569  15.3426  15.317       0.0          0.0
 15.3219  15.3247  15.3234  15.317   15.3027      0.0          0.0
 15.2758  15.282   15.2848  15.2835  15.2771  …   0.0          0.0
 15.2269  15.236   15.2422  15.2449  15.2437      0.0          0.0
 15.1756  15.1872  15.1962  15.2024  15.2051      0.0          0.0
 15.122   15.1359  15.1475  15.1565  15.1627      0.0          0.0
 15.0665  15.0825  15.0963  15.1079  15.1169      0.0          0.0
 15.0093  15.027   15.0429  15.0567  15.0683  …   0.0          0.0
 14.9504  14.9699  14.9876  15.0035  15.0173      0.0          0.0
 14.8901  14.9111  14.9305  14.9482  14.964       0.0          0.0
  ⋮                                           ⋱               
  0.0      0.0      0

In [19]:
size(rr)

(4096,)

In [20]:
size(wl)

(2049, 2049, 300)

### $W_l$ here is actually $C_l(r,r')$ that's the reason for its shape! we have 300 l values for 2049 r values which should run for all of the r and r' and thus will result into a (2049, 2049, 300) matrix.

In [21]:
size(ra)

(2049,)

## Everything that we've done so far was to convert the linear power spectrum of matter $P_m(k)$ to power spectrum in terms of bessel functions and spherical harmonics. "Spherically projected power spectrum."

**One can calculate it using the following:**

### $C_{l,matter}(r,r') = \frac{2}{\pi}\ \int k^2 dk j_l(kr) \ j_l^*(kr') P_m(k)$

The ultimate goal fo this notebook is to transform from $P(k)$ to $C_l(r)$. This reads in linear matter $P(k)$, and then computes $C_l(r_1,r_2)$ at $N$ points spaced logarithmically in $r$.  Then, using an assumed cosmology (including $h$), we convert that to binned $C_l(z_1,z_2)$ for galaxies, $C_l(d_1,d_2)$ for BH mergers, and $C_l(z_1,d_2)$ for the cross-spectra, for some assumed distance errors $\sigma_d$ for the BH mergers.

Having the matter power spectrum one can use the following relations to write the power spectrum for the Galaxy catalog and the gravitational wave events:

### $\delta_{Galaxy}(r) = b_{Galaxy}(r) \ \delta_m$
###  $\delta_{GW}(r) = b_{GW}(r) \ \delta_m$

and now having this one can write the auto-spectra for each of the catalogs and the cross-spectra respectively as follows:

## Auto-Spectra:

Writing from the scratch:
####  $<\delta n_{l' m',Galaxy}(r')^* \ \delta n_{lm, Galaxy}(r)>  \ =\  b_{Galaxy}(r')b_{Galaxy}(r) <\delta n_{l' m',matter}(r')^* \ \delta n_{lm, matter}(r)> 
\\  = b_{Galaxy}(r')b_{Galaxy}(r) \times \ \frac{1}{4\pi^4} \ \int dV_{k'}dV_{k} \ (-i)^{l'} (i)^{l} \ j_l(kr) \ j_{l'}^*(k'r') \ Y_l^m(\hat{k}) \ Y_{l'}^{m'*}(\hat{k}') \
\ \times <\delta \bar{n}_{matter}^*(k') \delta \bar{n}_{matter} (k)>
\\
= b_{Galaxy}(r')b_{Galaxy}(r) \times \frac{2}{\pi}\ \int k^2 dk j_l(kr) \ j_l^*(kr') P_{matter}(k) $

And then for both auto-spectras one can have:

### <span style="color: green;">$C_{l,galaxy}(r',r) \ = \ b_{Galaxy}(r')b_{Galaxy}(r) \times C_{l,matter}(r,r')$</span>

###  <span style="color: orange;">$C_{l,GW}(r',r) \ = \ b_{GW}(r')b_{GW}(r) \times C_{l,matter}(r,r')$</span>

## Cross-Spectra:

For cross spectra one can similarly calculate that:

### <span style="color: purple;">$C_{l,cross}(r',r) \ = \ b_{GW}(r')b_{Galaxy}(r) \times C_{l,matter}(r,r')$</span>


So one need to properly calculate the bias for both GW events and galaxy catalogs in order to make the cross spectra and Auto-spectra meaningful.
